# [문제]
- law_2.docx 파일을 읽고, Chroma 저장
- LLM에 질문 -> 답변
- '전제사기피해'에 관한 법률 질문만 받기
- 이 외의 질문을 하면 '답변을 할 수 없습니다.'

In [ ]:
# 필요한 모듈
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.messages import HumanMessage

## 환경변수 읽어오기
load_dotenv()
## 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)
## 임베딩 - > 벡터 데이터베이스에 저장, 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# ## 벡터 데이터베이스에 저장 [메모리]
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )
## 로컬에 저장된 임베딩 갖고 오기
database = Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

database
## 질문이 있으면, 벡터 데이터베이스에서 유사도 검색
query = '전세사기피해자로 인정받기 위한 조건은?'

## 유사도 검색으로 가져온 문서 LLM 전달
prompt = '''
[indentity]
- 너는 전세사기피해 법률 전문가이다
- [context]를 참고하여 사용자의 질문에 답변해 주세요

[context]
{retrieved_docs}

Question: {query}
'''

## LLM 모델 생성
llm = ChatOpenAI(model='gpt-4o')



In [8]:
## LLM 모델에 질문과 검색된 문서를 보냄
while True:
    query = input('전세사기피해에 대해 질문하세요 [종료하려면 S]')

    if query.upper()=='s':
        print('종료합니다')
        break

    if '전세사기피해' not in query:
        print('답변할 수 없습니다')
        continue

    ## 벡터 데이터베이스에서 유사도 검색
    retrieved_docs = database.similarity_search(query=query, k=3)
    
    formatted_prompt =  prompt.format(
    retrieved_docs=retrieved_docs,
    query=query
    )

    
    ## 프롬프트 변수에 값 설정
    response = llm.invoke([
        HumanMessage(content=formatted_prompt)
    ])
    print(response)

content='전세사기피해는 주로 임대차 계약과 관련된 사기 형태로, 임차인이 보증금을 떼이거나 손해를 보는 경우를 말합니다. 전세 사기는 다양한 방법으로 이루어질 수 있습니다. 예를 들어, 임대인이 실제로는 소유하지 않은 주택을 임대하는 경우, 주택에 설정된 담보나 채무를 숨기고 임차인을 속이는 경우, 혹은 전세 계약이 끝나기 전에 보증금을 돌려주지 않는 경우 등이 있습니다. 이러한 사기로 인해 임차인은 큰 금전적 손해를 입을 수 있으며, 경우에 따라 주거지를 잃는 심각한 상황에 처할 수 있습니다. 전세사기를 예방하기 위해서는 계약 전에 등기부등본을 확인하고, 반드시 공증 절차를 거치는 등의 주의가 필요합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 57, 'total_tokens': 247, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9bddfca6e2', 'id': 'chatcmpl-BeMlEjOnzLL0bXZb82qhRfgEMT3mc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--0684c200-23b7-47f4-afab-eb2f7a44b863-0' usage_metadata={'input_tokens': 57, 'output_tokens': 190, 'total_

KeyboardInterrupt: Interrupted by user